In [2]:
import json
import numpy as np

In [3]:
# !python -m pip install --upgrade pip
# !python -m pip install numpy 
# !python -m pip install json

In [4]:
# Читаем данные
with open("data.json", "r", encoding="utf8") as f:
    js = json.load(f)  

In [5]:
# Проверяем данные
js['spam'][3]['text_msg']

'Не пропустите! Распродажа билетов на концерт [название группы] со скидкой 50%! Поторопитесь, количество ограничено! https://www.kaspersky.ru/resource-center/threats/beware-online-dating-scams'

In [6]:
# Обучающая выборка (все кроме 10 последних)
spam_msg_ex = [row["text_msg"] for row in js["spam"][:-10]]
nospam_msg_ex = [row["text_msg"] for row in js["nospam"][:-10]]

# Тестовая выборка (10 последних)
spam_msg_test = [row["text_msg"] for row in js["spam"][-10:]]
nospam_msg_test = [row["text_msg"] for row in js["nospam"][-10:]]

In [7]:
# Создаем словарь, который содержит слово и сколько раз оно встречалось в наборе данных:
# пример spam = {'привет': 4, ...}, nospam = {'привет': 29, ...}
from collections import Counter 
def create_dict_from_array(array_):
    words = [word.lower() for row in array_ for word in row.split()]
    return Counter(words)

spam = create_dict_from_array(spam_msg_ex)
nospam = create_dict_from_array(nospam_msg_ex)

In [11]:
# Просмотреть результат
print([r for r in spam.items()][:5])

[('привет!', 4), ('вы', 4), ('выиграли', 1), ('iphone', 1), ('14', 1)]


$A$ - письмо спам

$B$ - письмо содержит конкретный текст (набор слов). Например письмо с текстом "Привет мир" и "мир привет" - это одно и тоже событие, а 
"привет мир" и "приветик мир" - это разные события.

$$P(A|B)P(B) = P(B|A)P(A)$$
$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

$$P(B) = P(B|A)P(A) + P(B|\overline{A})P(\overline{A})$$
$$P(A|B) = \frac{P(B|A)P(A)}{P(B|A)P(A) + P(B|\overline{A})P(\overline{A})}$$
$$P(\overline{A})=P(A)=0.5$$

Вероятность текста присутствовать в наборе данных "спама" и "не спама". 
$$P(B|A) = \prod_{i=1}^{m} P(W_i|A)$$
$$P(B|\overline{A}) =\prod_{i=1}^{m} P(W_i|\overline{A})$$
$m$ - число слов в сообщение из события $B$

вероятность содержания слова $W_i$ 
$$P(W_i|A) = \frac{N_A}{N_A+N_{\overline A}}$$
$$P(W_i|\overline{A}) = 1 - P(W_i|A)$$
$N_A$ - число слов в наборе спам

$N_\overline{A}$ - число слов в наборе не спам


In [20]:
def test_spam(text, w=0.5, debug=False):
    words = [word.lower() for word in text.split()]
    P = []
    for word in words:
        count = spam[word] + nospam[word]
        p = spam[word]/(count) if count != 0 else 0.5    
        P.append(p)
    
    PS = np.prod(P)
    PH = np.prod([1-p for p in P])
    rez_P = PS/(PS+PH) if PS+PH != 0 else 0
    if debug:
        print(P)
        print("P(B|A):", PS, "\nP(B|no A):", PH, "\nP(A|B):", rez_P)
        return float(rez_P)
    return 1 if rez_P > w else 0

In [21]:
y1 = [test_spam(t) for t in spam_msg_test]
print(y1)

[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]


In [22]:
y2 = [test_spam(t) for t in nospam_msg_test]
print(y2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


$$\begin{pmatrix} & \text{спам}  &  \text{не спам}\\ \text{определил как спам} & 0.9 & 0.1 \\\text{определил как не спам} & 0.1 & 0.9 \end{pmatrix}$$

In [23]:
# Ошибки появляются из-за того, что наш набор слишком маленький 

In [24]:
test_spam(spam_msg_test[-3], debug=True)
spam_msg_test[-3]

[1.0, 1.0, 0.5, 0.6153846153846154, 0.5, 0.25, 0.5, 0.38461538461538464, 0.5128205128205128, 0.0, 1.0]
P(B|A): 0.0 
P(B|no A): 0.0 
P(A|B): 0


'Ремонт без забот и хлопот. Мы берем все на себя! {email}'

In [25]:
test_spam(nospam_msg_test[1], debug=True)
nospam_msg_test[1]

[0.0, 1.0, 0.5128205128205128, 0.5]
P(B|A): 0.0 
P(B|no A): 0.0 
P(A|B): 0


'Как всегда на связи!'

In [31]:
def test_spam_v2(text, w=0.55, debug=False):
    words = [word.lower() for word in text.split()]
    P = []
    for word in words:
        count = spam[word] + nospam[word]
        p = spam[word]/(count) if count != 0 else 0.5    
        eps = 0.001
        p = p - eps if p > 0.5 else p+eps # if p!= 0.5 else 0.5
        P.append(p)
    
    PS = np.prod(P)
    PH = np.prod([1-p for p in P])
    rez_P = PS/(PS+PH) if PS+PH != 0 else 0
    if debug:
        print(P)
        print("P(B|A):", PS, "\nP(B|no A):", PH, "\nP(A|B):", rez_P)
        return float(rez_P)
    return 1 if rez_P > w else 0

In [32]:
y1 = [test_spam_v2(t) for t in spam_msg_test]
print(y1)
y2 = [test_spam_v2(t) for t in nospam_msg_test]
print(y2)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [35]:
# Теперь можно использовать этот фильтр для своих нужд :)

In [36]:
test_spam_v2('Привет! Ремонт')

1

In [37]:
test_spam_v2('Привет')

0